# Day 3a
In this notebook we will explore zero-shot and few-shot classification using a pre-trained LLM in the context of political bias classification. The goal will be to classify the tweets as either `"neutral"` or `"partisan"` using different classification techniques (zero-shot, few-shot, and feature extraction).  We will make use of a [dataset](https://data.world/crowdflower/classification-of-pol-social) of tweets containing four columns:

1. `"author"`: The author of the tweet.
2. `"text"`: The text of the tweet.
3. `"bias"`:  The political bias of the tweet. This can be either `"neutral"` or `"partisan"`.
4. `"type"`:  The type of tweet.

By the end of this notebook, you will be able to:
- Load a pre-trained causal LLM for text generation and run it on the GPU. 
- Implement zero-shot and few-shot classification and understand the difference between the two.

## Environment Setup
**Make sure to set your runtime to use a GPU by going to `Runtime` -> `Change runtime type` -> `Hardware accelerator` -> `T4 GPU`**

In [ ]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Installing requisite packages
    !pip install transformers accelerate &> /dev/null

    # Change working directory to day_3
    %cd /content/drive/MyDrive/LLM4BeSci_GSERM2024/day_3

In [ ]:
import pandas as pd
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch
import seaborn as sns
from tqdm.notebook import tqdm_notebook as tqdm

## Zero-shot Classification
We begin by loading the dataset as a `pandas.DataFrame`:

In [ ]:
media_bias_test = pd.read_csv('media_bias_test.csv')
media_bias_test

The code next loads the causal LLM and its corresponding tokenizer. We will use [`"microsoft/Phi-3-mini-128k-instruct"`](https://huggingface.co/microsoft/Phi-3-mini-128k-instruct), a recent model trained by Microsoft which shows impressive performance given its relatively small size. The smaller size has the main advantage that it can be run on the freely available GPUs on Google Colab. 

The code begins by setting the random seed. This helpe to ensure the reproducibility of the often stochastic processes involved in training and running LLMs. The code then loads the model and tokenizer. The model is loaded onto the GPU via `device_map="cuda"` and the model is set to use half-precision via `torch_dtype=torch.float16` to save memory (RAM). The `trust_remote_code=True` argument is used to trust the remote code, and `attn_implementation='eager'` is used for faster inference on the T4 GPUs available on Google Colab.

In [ ]:
torch.random.manual_seed(42) # For reproducibility

# Load the model 
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda", # Use GPU
    torch_dtype=torch.float16, # Use half-precision
    trust_remote_code=True, 
    attn_implementation='eager' # For faster inference on T4 GPUs
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

The code next initializes a `transformers` pipeline for text generation. This is a high-level API that allows for easy text generation using the pre-trained models. We will use this pipeline to classify the tweets as either `"neutral"` or `"partisan"` using zero-shot classification. The pipeline takes two arguments: 

1. `model`: The model to use for text generation.
2. `tokenizer`: The tokenizer to use for text generation.

In [ ]:
# Initialize pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Since we only want the model to output either `"neutral"` or `"partisan"`, the code provides a hard constraint on the generation by setting `"max_new_tokens": 10` and `"do_sample": False` in the `generation_args` dictionary, which is later feed into the text generation `pipe`. As a final piece of setup, we also add a soft constraint by prompting the model to `"Strictly answer with only 'neutral' or 'partisan'"`:

In [ ]:
# Text generation arguments
generation_args = {
    "max_new_tokens": 10,
    "return_full_text": False,
    "do_sample": False,
}

zero_shot_prompt = "Is this text neutral or partisan? Strictly answer with only 'neutral' or 'partisan':\n"

The code then iterates through each tweet in the test set, and generates a classification label using the zero-shot prompt. For each tweet, it:

1. Concatenates the zero-shot prompt with the tweet text.
2. Passes the concatenated text to the pipeline in the required json format.
3. Extracts the generated text from the output by checking for the presence of `"neutral"` or `"partisan"`.

In [ ]:
zero_shot_labels = []
for tweet in tqdm(media_bias_test['text']):
    # Concatenate zero-shot prompt with tweet
    message = [{"role": "user", "content": zero_shot_prompt + tweet}]  
    
    # Generate text and access output at index 0 at key 'generated_text'
    output = pipe(message, **generation_args)[0]['generated_text'].lower() 
    
    # Extract label and append to list
    label = 'neutral' if 'neutral' in output else 'partisan' if 'partisan' in output else 'nan' # 
    zero_shot_labels.append(label)

media_bias_test['zero_shot_label'] = zero_shot_labels
media_bias_test

Prediction accuracy can now be evaluated by comparing the predicted labels with the actual labels:

In [ ]:
# Comparing zero-shot and actual labels
print(f'Zero-shot accuracy: {(media_bias_test["zero_shot_label"] == media_bias_test["bias"]).mean()}')

The results can also visualized with a confusion matrix:

In [ ]:
# Confusion matrix
confusion = pd.crosstab(media_bias_test['bias'], media_bias_test['zero_shot_label'])
sns.heatmap(confusion, annot=True)

## Few-shot Classification
We will now perform few-shot classification using the same dataset. Few-shot classification is a form of in-context learning where a model is trained on a small number of examples from a new task. 

The code begins by taking 10 examples from the test set, 5 of each class. It will then use them as prompts to classify the remaining tweets. We will follow the same steps as in zero-shot classification, but with a few-shot adaptation:

In [ ]:
# Defining the number of examples (shots)
n_shots = 10

# Taking equal number of neutral and partisan examples
stratified_examples = pd.concat([media_bias_test.iloc[:n_shots//2], media_bias_test.iloc[-n_shots//2:]], )

# Shuffling the examples so that the model does not learn an arbitrary order
stratified_examples = stratified_examples.sample(frac=1, random_state=42)

# Assigning the few-shot prompt
few_shot_prompt = (
        "Based on the following texts and labels:\n\n" +
        '\n'.join(('Text: ' + stratified_examples['text'] + '\nLabel: ' + stratified_examples['bias']).iloc[:n_shots] + '\n') +
        "\n\nClassify the following texts. Strictly answer with only the labels 'neutral' or 'partisan':\n\n"
)

print(few_shot_prompt)

The code next classifies the remaining tweets using the few-shot prompt:

In [ ]:
# Editing headlines to include few-shot prompt
media_bias_test['few_shot_text'] = few_shot_prompt + media_bias_test['text']

# Classify all news articles
few_shot_label = []
for tweet in tqdm(media_bias_test['few_shot_text'].iloc[n_shots:]):
    message = [{"role": "user", "content": tweet}]
    
    # Generate text and access output at index 0 at key 'generated_text'
    output = pipe(message, **generation_args)[0]['generated_text'].lower()
    
    # Extract label and append to list
    label = 'neutral' if 'neutral' in output else 'partisan' if 'partisan' in output else 'nan'
    few_shot_label.append(label)

# Add few-shot labels to dataframe
media_bias_test['few_shot_label'] = ['nan'] * n_shots + few_shot_label
media_bias_test

The code now evaluates the few-shot classification accuracy by comparing the predicted labels with the actual labels. There are a couple of NaN labels that we will remove for convenience before calculating the accuracy (these would need to be investigated/imputed in a real-world scenario):

In [ ]:
# Removing any nan labels
media_bias_nan_removed = media_bias_test[media_bias_test['few_shot_label'] != 'nan']

# Calculating few-shot accuracy
few_shot_accuracy = (media_bias_nan_removed['few_shot_label'] == media_bias_nan_removed['bias']).mean()
print('Few-shot accuracy:', few_shot_accuracy)

The few-shot accuracy is slightly higher than the zero-shot accuracy. It is important to note that this will [not always be the case](https://dl.acm.org/doi/abs/10.1145/3411763.3451760). We can also visualize the confusion matrix:

In [ ]:
# Confusion matrix
confusion = pd.crosstab(media_bias_nan_removed['bias'], media_bias_nan_removed['few_shot_label'])
sns.heatmap(confusion, annot=True)

**Task 1:** Try editing the few-shot setting to include more examples (`n_shots=15`) and see how it affects the classification accuracy.
**Task 2:** Try editing the few-shot prompt to include more information about the task and see how it affects the classification accuracy. For instance, explain what the model should look out for in neutral versus partisan tweets.